In [1]:
import sys
from tqdm import tqdm
import pandas as pd
import numpy as np
from sklearn import datasets
import matplotlib
%matplotlib inline

import edward as ed
import tensorflow as tf
from edward.models import Bernoulli, MultivariateNormalTriL, Normal, OneHotCategorical
from edward.util import rbf
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from sklearn.preprocessing import OneHotEncoder

Using TensorFlow backend.


In [2]:
iris = datasets.load_iris()
df = pd.DataFrame(data=iris['data'], columns=iris['feature_names'])
i = pd.Series(iris['target'])
names = i.map(lambda x: iris['target_names'][x])
df['index'] = i
df['category_name'] = names
df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),index,category_name
0,5.1,3.5,1.4,0.2,0,setosa
1,4.9,3.0,1.4,0.2,0,setosa
2,4.7,3.2,1.3,0.2,0,setosa
3,4.6,3.1,1.5,0.2,0,setosa
4,5.0,3.6,1.4,0.2,0,setosa
5,5.4,3.9,1.7,0.4,0,setosa
6,4.6,3.4,1.4,0.3,0,setosa
7,5.0,3.4,1.5,0.2,0,setosa
8,4.4,2.9,1.4,0.2,0,setosa
9,4.9,3.1,1.5,0.1,0,setosa


In [3]:
# shuffle
df = df.sample(frac=1)

ys = df['index'].values
xs = df[['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']].values
x_train = xs[:100]
y_train = ys[:100]
x_test = xs[100:]
y_test = ys[100:]
print("Number of train data: {}".format(x_train.shape[0]))
print("Number of features: {}".format(x_train.shape[1]))

Number of train data: 100
Number of features: 4


In [4]:
encoder = OneHotEncoder(sparse=False)
y_train_ = encoder.fit_transform(y_train.reshape(-1, 1))
y_test_ = encoder.fit_transform(y_test.reshape(-1, 1))


model = Sequential()

model.add(Dense(10, input_shape=(4,), activation='relu', name='fc1'))
model.add(Dense(10, activation='relu', name='fc2'))
model.add(Dense(3, activation='softmax', name='output'))
# model.add(Dense(3, input_shape=(4, ), activation='softmax', name='output'))

optimizer = Adam(lr=0.001)
model.compile(optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

print('Neural Network Model Summary: ')
print(model.summary())

model.fit(x_train, y_train_, verbose=2, batch_size=5, epochs=30)

results = model.evaluate(x_test, y_test_)
results

Neural Network Model Summary: 
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
fc1 (Dense)                  (None, 10)                50        
_________________________________________________________________
fc2 (Dense)                  (None, 10)                110       
_________________________________________________________________
output (Dense)               (None, 3)                 33        
Total params: 193
Trainable params: 193
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/30
 - 0s - loss: 2.1558 - acc: 0.4500
Epoch 2/30
 - 0s - loss: 1.7023 - acc: 0.3500
Epoch 3/30
 - 0s - loss: 1.3276 - acc: 0.4200
Epoch 4/30
 - 0s - loss: 1.1309 - acc: 0.4900
Epoch 5/30
 - 0s - loss: 1.0264 - acc: 0.4700
Epoch 6/30
 - 0s - loss: 0.9604 - acc: 0.5200
Epoch 7/30
 - 0s - loss: 0.9266 - acc: 0.5700
Epoch 8/30
 - 0s - loss: 0.8909 - acc: 0.5000
Epoc

[0.383165602684021, 0.94]

In [5]:
N = 100
num_feature = 4
def neural_network(x):
    h = tf.nn.relu(tf.matmul(x, w1) + b1)
    h = tf.nn.relu(tf.matmul(h, w2) + b2)
    h = tf.nn.softmax(tf.matmul(h, w3) + b3)
    return h

with tf.name_scope("model"):
    w1 = Normal(loc=tf.zeros([num_feature, 10]), scale=tf.ones([num_feature, 10]), name="w1")
    w2 = Normal(loc=tf.zeros([10, 10]), scale=tf.ones([10, 10]), name="w2")
    w3 = Normal(loc=tf.zeros([10, 3]), scale=tf.ones([10, 3]), name="w3")
    b1 = Normal(loc=tf.zeros(10), scale=tf.ones(10), name="b1")
    b2 = Normal(loc=tf.zeros(10), scale=tf.ones(10), name="b2")
    b3 = Normal(loc=tf.zeros(3), scale=tf.ones(3), name="b3")

    x = tf.placeholder(tf.float32, [None, num_feature], name="x")
    y = OneHotCategorical(neural_network(x), name="y")
    

# INFERENCE
with tf.variable_scope("posterior"):
    with tf.variable_scope("qw1"):
        loc = tf.get_variable("loc", [num_feature, 10])
        scale = tf.nn.softplus(tf.get_variable("scale", [num_feature, 10]))
        qw1 = Normal(loc=loc, scale=scale)
    with tf.variable_scope("qw2"):
        loc = tf.get_variable("loc", [10, 10])
        scale = tf.nn.softplus(tf.get_variable("scale", [10, 10]))
        qw2 = Normal(loc=loc, scale=scale)
    with tf.variable_scope("qw3"):
        loc = tf.get_variable("loc", [10, 3])
        scale = tf.nn.softplus(tf.get_variable("scale", [10, 3]))
        qw3 = Normal(loc=loc, scale=scale)
    with tf.variable_scope("qb1"):
        loc = tf.get_variable("loc", [10])
        scale = tf.nn.softplus(tf.get_variable("scale", [10]))
        qb1 = Normal(loc=loc, scale=scale)
    with tf.variable_scope("qb2"):
        loc = tf.get_variable("loc", [10])
        scale = tf.nn.softplus(tf.get_variable("scale", [10]))
        qb2 = Normal(loc=loc, scale=scale)
    with tf.variable_scope("qb3"):
        loc = tf.get_variable("loc", [3])
        scale = tf.nn.softplus(tf.get_variable("scale", [3]))
        qb3 = Normal(loc=loc, scale=scale)
        
        
inference = ed.KLqp({w1: qw1, b1: qb1,
                   w2: qw2, b2: qb2,
                   w3: qw3, b3: qb3}, data={x: x_train, y: y_train_})
inference.run(n_iter=100000)

# with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
#     samples_num = 100
#     BATCH_SIZE = 10
#     for epoch in tqdm(range(1000), file=sys.stdout):
#         perm = np.random.permutation(N)
#         for i in range(0, N, BATCH_SIZE):
#             batch_x = x_train[perm[i:i+BATCH_SIZE]]
#             batch_y = y_train[perm[i:i+BATCH_SIZE]]
#             inference.update(feed_dict={x: batch_x, y: batch_y})

# ed.evaluate('categorical_accuracy', data={x: test_x, y: test_y})


/home/app-user/.pyenv/versions/3.6.3/lib/python3.6/site-packages/edward/util/random_variables.py:52: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  not np.issubdtype(value.dtype, np.float) and \


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

100000/100000 [100%] ██████████████████████████████ Elapsed: 107s | Loss: 123.637


In [6]:
y_post = ed.copy(y, {w1: qw1, b1: qb1,
                   w2: qw2, b2: qb2,
                   w3: qw3, b3: qb3})

In [7]:
ed.evaluate('categorical_accuracy', data={x: x_test, y_post: y_test_})


/home/app-user/.pyenv/versions/3.6.3/lib/python3.6/site-packages/edward/util/random_variables.py:52: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  not np.issubdtype(value.dtype, np.float) and \


0.32

In [8]:
y_samples = y_post.sample(10).eval(feed_dict={x: x_test})
for sample in y_samples:
    print(sample)

[[1 0 0]
 [0 1 0]
 [0 1 0]
 [1 0 0]
 [0 1 0]
 [0 0 1]
 [1 0 0]
 [1 0 0]
 [0 1 0]
 [1 0 0]
 [1 0 0]
 [0 0 1]
 [1 0 0]
 [0 0 1]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [0 0 1]
 [0 1 0]
 [0 1 0]
 [1 0 0]
 [0 0 1]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [0 0 1]
 [1 0 0]
 [0 0 1]
 [0 1 0]
 [0 0 1]
 [1 0 0]
 [1 0 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [0 1 0]
 [0 1 0]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [1 0 0]
 [0 0 1]
 [0 0 1]
 [1 0 0]]
[[1 0 0]
 [0 1 0]
 [1 0 0]
 [0 0 1]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [0 0 1]
 [0 0 1]
 [1 0 0]
 [0 1 0]
 [1 0 0]
 [0 0 1]
 [0 0 1]
 [1 0 0]
 [0 0 1]
 [1 0 0]
 [0 1 0]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [1 0 0]
 [1 0 0]
 [0 0 1]
 [1 0 0]
 [1 0 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [1 0 0]
 [1 0 0]
 [0 1 0]
 [1 0 0]
 [0 0 1]
 [0 0 1]
 [1 0 0]
 [0 1 0]
 [0 1 0]
 [1 0 0]
 [1 0 0]
 [0 0 1]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [0 1 0]
 [1 0 0]
 [1 0 0]
 [0 0 1]
 [1 0 0]]
[[0 0 1]
 [0 0 1]
 [0 1 0]
 [1 0 0]
 [0 0 1]
 [0 1 0]
 [1 0 0]
 [1 0 0]
 [0 0 1]
 [1 0 0]
 [1 0 0]

In [9]:
model.predict(x_test)

array([[4.56547205e-05, 3.02214324e-01, 6.97739959e-01],
       [2.42731461e-04, 2.72665471e-01, 7.27091789e-01],
       [7.02214020e-05, 3.36180896e-01, 6.63748860e-01],
       [4.05087339e-04, 3.77028108e-01, 6.22566760e-01],
       [4.66681144e-04, 3.54133517e-01, 6.45399749e-01],
       [1.12071575e-03, 4.06943738e-01, 5.91935515e-01],
       [3.06782895e-03, 4.52739984e-01, 5.44192195e-01],
       [1.57014874e-03, 4.62883383e-01, 5.35546362e-01],
       [1.99520332e-03, 4.35738474e-01, 5.62266290e-01],
       [9.88438042e-05, 2.36306608e-01, 7.63594568e-01],
       [8.16914981e-05, 2.14365423e-01, 7.85552859e-01],
       [1.32962933e-03, 4.08471078e-01, 5.90199292e-01],
       [1.03702820e-04, 2.81911522e-01, 7.17984736e-01],
       [1.60561787e-04, 3.34354997e-01, 6.65484488e-01],
       [4.84791351e-03, 4.58710909e-01, 5.36441207e-01],
       [1.66264456e-03, 4.62079108e-01, 5.36258280e-01],
       [2.57110252e-04, 2.89547831e-01, 7.10195065e-01],
       [2.96964520e-03, 4.21948